In [46]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, Input, Output, callback, html
import plotly.express as px
import yfinance as yf
import plotly.graph_objects as go



In [47]:
app = Dash()

In [48]:
app.layout = html.Div([

    html.H1(children='Stock Tracker', style={'textAlign':"center"}),

    html.Div([
        html.Label('Enter Stock Symbol (e.g. AAPL):'),
        dcc.Input(id='stock-input', type='text', value='AAPL')
    ], style={'margin': '20px'}),

    html.Div([
        html.Label("Select number of days:"),
        dcc.Slider(1,5,1, value=2, id="day-slider")
    ], style={'margin': '20px'}),

    dcc.Graph(id='candlestick-chart'),
    dcc.Graph(id='volume-chart')

])

In [49]:
@callback(
    Output('candlestick-chart', 'figure'),
    Output('volume-chart', 'figure'),
    Input('stock-input', 'value'),
    Input('day-slider', 'value')
)
def update_stock_chart(ticker, days):
    # Download minute-by-minute stock data from Yahoo Finance
    period = f'{days}d'
    interval = '1m'
    try:
        df = yf.download(tickers=ticker, period=period, interval=interval)
    except Exception:
        return px.line(title="Error loading data"), px.bar(title="Volume Unavailable")

    if df.empty:
        return px.line(title="No Data Found"), px.bar(title="No Volume Data")

    print(df.head)

    df.reset_index(inplace=True)  # Convert datetime to column

    # Moving Averages (on Close price)
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()

    # Candlestick + Moving Averages
    fig_price = go.Figure()

    # Candlestick trace
    fig_price.add_trace(go.Candlestick(
        x=df['Datetime'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
    ))

    # 30-min MA
    fig_price.add_trace(go.Scatter(
        x=df['Datetime'],
        y=df['MA_30'],
        mode='lines',
        name='30-min MA',
        line=dict(color='blue')
    ))

    # 200-min MA
    fig_price.add_trace(go.Scatter(
        x=df['Datetime'],
        y=df['MA_200'],
        mode='lines',
        name='200-min MA',
        line=dict(color='red')
    ))

    fig_price.update_layout(
        title=f'{ticker.upper()} Candlestick Chart with Moving Averages ({days} Day{"s" if days > 1 else ""})',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False,
        plot_bgcolor='white',
        legend_title='Legend'
    )

    # Volume bar chart
    fig_volume = px.bar(df, x='Datetime', y='Volume', title='Trading Volume')
    fig_volume.update_layout(
        xaxis_title="Time",
        yaxis_title="Volume",
        plot_bgcolor='white'
    )

    return fig_price, fig_volume

In [50]:
if __name__ == '__main__':
    app.run(debug=True)

/opt/miniconda3/lib/python3.12/site-packages/charset_normalizer/api.py:105: UserWarning:

Trying to detect encoding from a tiny portion of (23) byte(s).

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
/opt/miniconda3/lib/python3.12/site-packages/charset_normalizer/api.py:105: UserWarning:

Trying to detect encoding from a tiny portion of (23) byte(s).

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
